In [10]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random
import time
import string
import unicodedata
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
import multiprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import glob

[nltk_data] Downloading package stopwords to /home/lucas-
[nltk_data]     nunes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1. Carrega base de application

In [26]:
# df_application = pd.read_csv('/content/drive/MyDrive/Pós Tech/Tech Challenges/Tech Challenge 5/Dados/silver/dados_processed/application_processed.csv')
df_application = pd.read_csv('/home/lucas-nunes/workspace/Postech/challenges/5_data/data/silver/processed/application_processed.csv')
df_application.columns

Index(['job_id', 'objetivo_profissional', 'data_criacao', 'local',
       'sabendo_de_nos_por', 'data_atualizacao', 'codigo_profissional',
       'data_aceite', 'fonte_indicacao', 'telefone_celular', 'sexo',
       'estado_civil', 'pcd', 'endereco', 'titulo_profissional',
       'area_atuacao', 'conhecimentos_tecnicos', 'certificacoes',
       'outras_certificacoes', 'remuneracao', 'nivel_profissional',
       'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma',
       'cv_pt', 'instituicao_ensino_superior', 'cursos', 'ano_conclusao',
       'data_admissao', 'data_ultima_promocao', 'conhecimentos_tecnicos_list',
       'certificacoes_list', 'outras_certificacoes_list',
       'remuneracao_numeric', 'cv_pt_cleaned', 'telefone_celular_normalized'],
      dtype='object')

In [27]:
df_application.shape

(42482, 37)

# 2. Seleciona coluna do currículo

In [28]:
# Count words for each line
df_application['cv_pt_cleaned']


0        assistente administrativo santosbatista itapec...
1        formação acadêmica ensino médio (2º grau) em e...
2        objetivo: área administrativa | financeira res...
3        formação ensino médio completo informática int...
4        última atualização em 09/11/2021 ­ sp ensino s...
                               ...                        
42477                                                  NaN
42478                                                  NaN
42479                                                  NaN
42480                                                  NaN
42481                                                  NaN
Name: cv_pt_cleaned, Length: 42482, dtype: object

# 3. Limpeza do texto

In [29]:
# Create df only with the first 5 lines preserving the column name cv_pt_cleaned
# df_application_teste = df_application[:100]
df_application_teste = df_application

In [6]:
import spacy.cli
spacy.cli.download("pt_core_news_sm")
import spacy

# carregar modelo para português
nlp = spacy.load("pt_core_news_sm")

def remove_person_names(text: str) -> str:
    doc = nlp(text)
    return " ".join([token.text for token in doc if token.ent_type_ != "PER"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 43.3 MB/s  0:00:0040.7 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('rslp')

from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer

# inicializa stemmer
stemmer = RSLPStemmer()

def normalize_accents(text: str) -> str:
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def remove_punctuation(text: str) -> str:
    table = str.maketrans({key: " " for key in string.punctuation})
    return text.translate(table)

def normalize_str(text: str) -> str:
    text = text.lower()
    text = re.sub(r"\d+", " ", text)           # remove números
    text = remove_punctuation(text)            # remove pontuação
    text = normalize_accents(text)             # remove acentos
    text = re.sub(r"\s+", " ", text).strip()   # normaliza espaços
    return text

def tokenizer(text: str):
    stop_words_br = set(nltk.corpus.stopwords.words("portuguese"))
    #stop_words_en = set(nltk.corpus.stopwords.words("english"))
    if isinstance(text, str):
        text = normalize_str(text)                                              # normaliza string
        text = remove_person_names(text)                                        # remove nomes
        tokens = word_tokenize(text, language="portuguese")                     # tokeniza para a lingua portuguesa
        tokens = [t for t in tokens if t not in stop_words_br and len(t) > 2]
        #tokens = [t for t in tokens if t not in stop_words_en and len(t) > 2]
        tokens = [stemmer.stem(t) for t in tokens]                              # stemiza tokens
        return tokens
    return None

[nltk_data] Downloading package punkt to /home/lucas-
[nltk_data]     nunes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/lucas-
[nltk_data]     nunes/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package rslp to /home/lucas-nunes/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [32]:
df_application_teste['cv_tokenizado'] = df_application_teste['cv_pt_cleaned'].apply(tokenizer)

In [36]:
df_application_teste = df_application_teste[['cv_pt_cleaned', 'cv_tokenizado']]

In [38]:
df_application_teste.to_parquet('/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_token_2.parquet')

In [3]:
df_application_teste = pd.read_parquet('/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_token_2.parquet')

In [4]:
df_application_teste

,cv_pt_cleaned,cv_tokenizado
0,assistente administrativo santosbatista itapec...,"[assist, administr, santosbat, itapecer, serr,..."
1,formação acadêmica ensino médio (2º grau) em e...,"[formaca, academ, ensin, medi, grau, ensin, me..."
2,objetivo: área administrativa | financeira res...,"[obje, are, administr, financ, resum, profiss,..."
3,formação ensino médio completo informática int...,"[formaca, ensin, medi, complet, informa, inter..."
4,última atualização em 09/11/2021 ­ sp ensino s...,"[ult, atualizaca, ensin, superi, administraca,..."
...,...,...
42477,None,None
42478,None,None
42479,None,None
42480,None,None


In [9]:
def tokenize_and_vectorize(df, tokenizer, filename_prefix, batch_idx):
    vectorizer = TfidfVectorizer(tokenizer=tokenizer)
    X = vectorizer.fit_transform(df["cv_pt_cleaned"].fillna(""))
    df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    df_tfidf.to_parquet(f"{filename_prefix}_{batch_idx}.parquet")
    return df_tfidf

batch_size = 1000
filename_prefix = '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vector'

for i in range(0, len(df_application_teste), batch_size):
    batch_df = df_application_teste.iloc[i:i+batch_size]
    tokenize_and_vectorize(batch_df, tokenizer, filename_prefix, i // batch_size)
    print(f'end iteration: {i // batch_size}')

/home/lucas-nunes/workspace/Postech/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


end iteration: 0


/home/lucas-nunes/workspace/Postech/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


end iteration: 1


/home/lucas-nunes/workspace/Postech/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


...

end iteration: 40


/home/lucas-nunes/workspace/Postech/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


end iteration: 41


/home/lucas-nunes/workspace/Postech/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


end iteration: 42


In [ ]:
first = True
with open('/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vector_all.csv', 'w') as f_out:
    for file in list_files_vector:
        df_tmp = pd.read_parquet(file)
        df_tmp.to_csv(f_out, header=first, index=False)
        first = False

ValueError: Column names of new data are ['abac', 'abaix', 'abap', 'abastec', 'abb', 'abengo', 'abert', 'ability', 'abnamr', 'abr', 'abrang', 'abril', 'absorv', 'ac', 'aca', 'acab', 'academ', 'academy', 'accentur', 'acces', 'accounting', 'aceit', 'acert', 'acess', 'ach', 'acid', 'acion', 'aco', 'acom', 'acompanh', 'acord', 'acostum', 'acresc', 'acrobat', 'act', 'acting', 'actiom', 'action', 'activ', 'activat', 'activiti', 'acuc', 'acumul', 'acunetix', 'ad', 'adab', 'adapt', 'adaptaca', 'add', 'addvi', 'adequ', 'adequaca', 'adequaco', 'ader', 'aderenc', 'adf', 'adic', 'adm', 'admin', 'administering', 'administr', 'administra', 'administraca', 'administration', 'admissa', 'ado', 'adob', 'adoca', 'adopt', 'adot', 'adp', 'adquir', 'adsl', 'advanced', 'advers', 'advi', 'advpl', 'adword', 'ae', 'aere', 'aeronav', 'aerospac', 'aethr', 'aeve', 'af', 'aferica', 'aff', 'affinity', 'afili', 'afim', 'afric', 'aft', 'age', 'agel', 'agenc', 'agend', 'agent', 'agf', 'agil', 'agiliz', 'ago', 'agost', 

In [14]:
list_files_vector

['/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vector/talent_pool_vector_12.parquet',
 '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vector/talent_pool_vector_2.parquet',
 '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vector/talent_pool_vector_13.parquet',
 '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vector/talent_pool_vector_3.parquet',
 '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vector/talent_pool_vector_0.parquet',
 '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vector/talent_pool_vector_31.parquet',
 '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vector/talent_pool_vector_6.parquet',
 '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/talent_pool_vector/talent_pool_vector_33.parquet',
 '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/gold/ta

In [ ]:
# Calcula a matriz de similaridade do cosseno
cosine_sim = linear_kernel(df_tfidf, df_tfidf)

In [ ]:
cosine_sim

array([[1.        , 0.09650452, 0.32610838, ..., 0.04060293, 0.01643154,
        0.03483504],
       [0.09650452, 1.        , 0.20202968, ..., 0.07989363, 0.02201943,
        0.07469299],
       [0.32610838, 0.20202968, 1.        , ..., 0.14084257, 0.01658457,
        0.09780909],
       ...,
       [0.04060293, 0.07989363, 0.14084257, ..., 1.        , 0.01148794,
        0.03139983],
       [0.01643154, 0.02201943, 0.01658457, ..., 0.01148794, 1.        ,
        0.21295057],
       [0.03483504, 0.07469299, 0.09780909, ..., 0.03139983, 0.21295057,
        1.        ]], shape=(100, 100))